### Import Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
from sbi.inference import SNPE
from sbi import utils as utils
from sbi.analysis import run_sbc, sbc_rank_plot
from astropy.io import fits
from astropy.table import Table, Column
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import pickle

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/home/tingli/.conda/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Loading

In [2]:
filename = "/raid/users/heigerm/catalogues/sp_x_apogee_x_spspectra_rvtab.fits"
# sp data
HDUlist = fits.open(filename)
# DESI
sp_tab = Table(HDUlist['SPTAB'].data)   
# APOGEE
apogee_tab = Table(HDUlist['APOGEEDR17'].data) 
# DESI SP Spectra
spectra = Table(HDUlist['SPECTRA_SP'].data)

### Data pre-processing

In [3]:
# Define parameters
targets = ['FE_H', 'MG_FE', 'C_FE', 'O_FE', 'CI_FE', 'AL_FE', 'SI_FE', 'K_FE', 'CA_FE', 'MN_FE', 'NI_FE', 'LOGG', 'TEFF']
feh_target, mgfe_target, cfe_target, ofe_target, cife_target, alfe_target, sife_target, kfe_target, cafe_target, mnfe_target, nife_target, log_g, temperature = (np.array(apogee_tab[col]) for col in targets)
targets_arr = [feh_target, kfe_target, cfe_target, cafe_target, nife_target, mnfe_target, ofe_target, cife_target, alfe_target]
# check for Al error = 0 case
alfe_target_err = np.array(apogee_tab['AL_FE_ERR'])
abnormal_rows = np.unique([index for target in targets_arr for index, value in enumerate(target) if value > 10] + 
                          [index for index, value in enumerate(alfe_target_err) if value == 0])


# Mask the abnormal rows across relevant datasets
mask = ~np.isin(np.arange(len(apogee_tab)), abnormal_rows)
apogee_tab_masked = apogee_tab[mask]
spectra_masked = spectra[mask]
sp_tab_masked = sp_tab[mask]

# Reconstruct target arrays with masked data
target_values_masked = {target: np.array(apogee_tab_masked[target]) for target in targets}

In [4]:
len(spectra_masked)

7336

### Spectra

In [5]:
# Combine spectra from the three arms and normalize
gb_combined_spectra = Table(names=['combined_flux', 'combined_wavelength'], dtype=['object', 'object'])

for row in spectra_masked:
    # Combine and sort flux and wavelength from all arms
    combined_flux = np.concatenate([row['flx_B'], row['flx_R'], row['flx_Z']])
    combined_wavelength = np.concatenate([row['B_WAVELENGTH'], row['R_WAVELENGTH'], row['Z_WAVELENGTH']])
    sort_order = np.argsort(combined_wavelength)
    combined_flux, combined_wavelength = combined_flux[sort_order], combined_wavelength[sort_order]

    # Normalize flux
    global_median = np.median(combined_flux)
    IQR = np.percentile(combined_flux, 75) - np.percentile(combined_flux, 25)
    normalized_flux = (combined_flux - global_median) / IQR

    gb_combined_spectra.add_row([normalized_flux, combined_wavelength])

### Input data

In [6]:
flux = np.array(gb_combined_spectra['combined_flux'])
# Input spectra
X = np.array([np.array(flux_val, dtype=float) for flux_val in flux])
# Parameters
theta = np.column_stack([target_values_masked[target] for target in targets])

In [7]:
print("Length of X:", len(X))
print("Dimensions of X:", len(X[0]))

Length of X: 7336
Dimensions of X: 13787


### Model Training (Cross-Validation)

In [ ]:
num_folds = 5

kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Define the neural network structure for embedding
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(13787, 4000),
            nn.GELU(),
            nn.Linear(4000, 2000),
            nn.GELU(),
            nn.Linear(2000, 1000),
            nn.GELU(),
            nn.Linear(1000, 500),
            nn.GELU(),
            nn.Linear(500, 100),
            nn.GELU(),
            nn.Linear(100, 50))

    def forward(self, x):
        return self.model(x)

# Iterate through the folds
for fold, (train_index, test_index) in enumerate(kf.split(X, theta)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = theta[train_index], theta[test_index]

    # Standardize the data
    scaler = StandardScaler().fit(X_train)
    X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)

    # Convert to PyTorch tensors
    X_train, X_test, y_train, y_test = map(torch.Tensor, (X_train, X_test, y_train, y_test))


    # Initialize the neural posterior with the defined model
    neural_posterior = utils.posterior_nn(model="nsf", embedding_net=Model(), hidden_features=50, num_transforms=5)
    inference = SNPE(density_estimator=neural_posterior)
    inference.append_simulations(y_train, X_train)

    # Train the density estimator and build posterior
    density_estimator = inference.train()
    posterior = inference.build_posterior(density_estimator)
            
    # Save the model
    model_pkl_file = f"SBI_fold_{fold}.pkl" 

    with open(model_pkl_file, 'wb') as file:  
        pickle.dump(posterior, file)

 Training neural network. Epochs trained: 31 60 epochs.